# Mapping the Boulevard BID

This is a simple workflow to combine data sets and display them on the map.  I'm using a basic ipyleaflet control for display. Spatial analysis is via shapely/geopandas.<br/>
The data comes from City of San Diego and the previously processed business data set.<br/><br/>

**Steps:**

  1. Display the basic, `background map`
  
  2. Add the Business Improvement Districts `(BIDs) polygons` as map overlay
  
     - Quick look at what's in the data
     
     - Create overlay for the map using Boulevard BID
     
  3. Add `business data` to the map as `overlay`
  
     - Use geocoded shape files from wrangling.ipynb
     
     - Remember the data is from two zip codes, 92115 and 92116
     
     - Add markers to the geodataframe (**gdf**) and display as overlay
     
  4. Display `BID specific` businesses as `overlay`
  
     - Apply filter to get businesses within BID boundary as new gdf
     
     - Add markers to the new gdf and display as overlay
  
  5. Summary and next steps
  



# Display basic map

This is a simple example using ipyleaflet, various ipywidgets, and the `OSM base maps` as ESRI tiles.<br/>

As we work through the steps in this notebook we will be adding overlays to the map to select map elements.

In this section we add the layers to select image or map backgrounds.

In [ ]:
import geopandas as gpd
from ipywidgets import HTML, Layout
from ipyleaflet import (Map, Rectangle, GeoJSON,
                        MarkerCluster, GeoData, LayersControl,
                        LayerGroup, Marker, WidgetControl,
                        CircleMarker,
                       basemaps, basemap_to_tiles)

from tqdm import tqdm

In [ ]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True


map_display = Map(center=(32.715, -117.1625), zoom=12,
                  layers=[imagery, osm],
                  layout=Layout(height="700px"),
                  scroll_wheel_zoom=True)

map_display.add_control(LayersControl())
map_display

# BID Polygons

Previous notebooks looked at the business data set.  Now I'd like to add some `context` to that data.<br/>
The first source to look at is the [business improvement districts](https://www.sandiego.gov/economic-development/about/bids).  BIDs are designed to help small businesses so that seems like a good fit!

Since this is the first time we've seen this data we should do a standard analysis of what's included.<br/>

Note we'll want to have an understanding so we can select individual BIDs.

In [ ]:
bids_gdf = gpd.read_file("../data/bids/bids_datasd.shp")

bids_gdf = bids_gdf.to_crs(epsg=4326)

In [ ]:
bids_gdf.info()

bids_gdf is a **gdf** from geopandas.<br/>
So info() says all the columns have values. <br/>
Next look at the data.  Notice the columns that will help: name, geometry.  We might be able to use link too?

In [ ]:
bids_gdf

This is the first example of adding a **gdf** to the map.  This too is a fairly standard idiom.

In [ ]:
bids = GeoData(geo_dataframe = bids_gdf,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'BIDs')

map_display += bids

This is one (standard) approach to displaying info about elements of an ipyleaflet map.<br/>
Note the info is displayed in the upper corner.<br/>
I'm not real partial to this approach but I haven't yet found a way to do tooltip like popups?

Does anyone want to add the url?

In [ ]:
bid_html = HTML('''Hover over a district''')
bid_html.layout.margin = '0px 20px 20px 20 px'
bid_control = WidgetControl(widget=bid_html, position='topright')

def update_bid_html(feature, **kwargs):
    bid_html.value = f"<b>{feature['properties']['name']}"
    
map_display.add_control(bid_control)  # does += work for this?

bids.on_hover(update_bid_html)

### Go back and look at the map.

Extra credit if you know how display the map so you don't have to go back and forth!

# Add Business Data

First, we'll add the data from two zip codes, 92115 and 92116 produced in the wrangling.ipynb.<br/>

Remember this zip code based approach is a workaround to use the Nominatim geocoder.<br/>
I wanted to have smaller data sets to geocode because of the 1 second delay for each request.<br/>
Once I've installed a local version of Nominatim and geocoded the entire set you can do this different.

In [ ]:
multi_zip_gdf = gpd.read_file("../data/ecb.shp").reset_index()

In [ ]:
len(multi_zip_gdf)

In [ ]:
multi_zip_gdf.info()

Don't forget that Nominatim doesn't always give you a Point!<br/>

We do get pretty good results in the set of data though.

In [ ]:
valid_geo_gdf = multi_zip_gdf.dropna(subset=['geometry']).reset_index()

all_biz_count = len(multi_zip_gdf)
geocoded_biz_count = len(valid_geo_gdf)
missing_geocode_count = all_biz_count - geocoded_biz_count
#len(multi_zip_gdf) - len(valid_geo_gdf)

print(f"Unable to geocode {missing_geocode_count} businesses out of {all_biz_count} - {missing_geocode_count/all_biz_count:.2%}")

## Add business overlay

This is another standard idiom.  I'm using CircleMarkers with default colors and then displayed as MarkerCluster.<br/>

This is another place that we could devise a color scheme for the markers?  Maybe the ones in our sector get a different color?...or...<br/>

Another extension would be the type of information displayed when selecting a marker.  For now it's just the DBA column.

In [ ]:
business = list()

for i, r in tqdm(valid_geo_gdf.iterrows()):
    marker = CircleMarker(location=(r.geometry.y, r.geometry.x), radius=5, stroke=False, fill_color="blue", fill_opacity=1.0)
    msg = HTML()
    msg.value = f"{r['DBA NAME']}"
    marker.popup = msg
    business.append(marker)
    r['marker'] = marker

In [ ]:
business_cluster = MarkerCluster(markers=business, name='Businesses')
map_display.add_layer(business_cluster)

### Go back and look at the map.

## El Cajon BID Businesses

Can get this from the bids_gdf.<br/>

Look at the bids_bid again if needed.  I'm using simple query to get the row by name.<br/>

What I need for spatial filtering is the geometry.

In [ ]:
#i.e.
ecb_bid = bids_gdf.query(f"name == 'El Cajon Boulevard Central'").reset_index()

In [ ]:
ecb_bid

## Filter based on the polygon

In [ ]:
ecb_biz_gdf = valid_geo_gdf[valid_geo_gdf.geometry.within(ecb_bid.iloc[0].geometry)]

In [ ]:
len(ecb_biz_gdf)

In [ ]:
ecb_biz_gdf.columns

In [ ]:
ecb_biz_gdf = ecb_biz_gdf.drop(columns=['level_0']).reset_index()

So ecb_biz_gdf is the set of businesses within the boundary of the Blvd BID.<br/>

Not 100% sure if this is how members are measured, but ...

## Create the second set up businesses for the map

Second time we've seen this idiom.

In [ ]:
ecb_business = list()
for i, r in tqdm(ecb_biz_gdf.iterrows()):
    marker = CircleMarker(location=(r.geometry.y, r.geometry.x), radius=5, stroke=False, fill_color="blue", fill_opacity=1.0)
    msg = HTML()
    msg.value = f"{r['DBA NAME']}"
    marker.popup = msg
    ecb_business.append(marker)
    r['marker'] = marker

In [ ]:
ecb_business_cluster = MarkerCluster(markers=ecb_business, name='Boulevard Businesses')
map_display.add_layer(ecb_business_cluster)

### Now go back and look at the map again.

By selecting the different overlays you can see how the elements relate.<br/>

Hoover works for the BID poly's and mouse clicks on the business markers reports the DBA NAME.

## This demonstrates the basics

With this we have basic tools to put businesses on a map, display polygons, spatial filtering, ...  Basics